In [2]:
import torch_geometric.transforms
from torch_geometric.nn import knn_graph
import torch_geometric.data
import torch 
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import numpy as np
import pickle
from torch_geometric.data import DataLoader
import linecache
import random

In [2]:
pickle_in2 = open("data/stdc/d9p1/dict.eq_distr","rb")
eq_distr = []
while 1:
    try:
        eq_distr.append(pickle.load(pickle_in2))
    except EOFError:
        break

print(len(eq_distr))

13936


In [3]:
#Use files from "new_datacreator"

node_feat_load = np.genfromtxt('data/stdc/d9p1/node_features.txt')
edge_idx_load = np.genfromtxt('data/stdc/d9p1/edgelist.txt',usecols=(0,1))
edge_feat_load = np.genfromtxt('data/stdc/d9p1/edgelist.txt',dtype = float,usecols=3,comments = '}')
graph_info = np.genfromtxt('data/stdc/d9p1/graph_info.txt')

In [4]:
graphs = []

#could probably make it more elegant but for now the holder indices works fine i guess :P
holder=0
edge_holder =0
for ix in range(len(graph_info)):
#for ix in range(5):

    num_nodes = int(graph_info[ix])
    edge_amount = int(num_nodes*(num_nodes-1))
    
    node_feat = node_feat_load[holder:holder+num_nodes]
    edge_idx = edge_idx_load[edge_holder:edge_amount+edge_holder]
    edge_feat = edge_feat_load[edge_holder:edge_amount+edge_holder]
    
    #print(node_feat)
    #print(edge_idx)
    #print(edge_feat)
    
    node_features = torch.from_numpy(node_feat)
    edge_features = torch.from_numpy(edge_feat)
    
    edge_index = torch.from_numpy(np.transpose(edge_idx)).type(torch.LongTensor)
    #edge_index = torch_geometric.utils.to_undirected(edge_index)
    
    #print(node_features)
    #print(edge_index)
    #print(edge_features.shape)
    eq_class = np.array(np.around(eq_distr[ix]*0.01,decimals=3))#should change this to float with prec.3 or something
    eq_class = torch.from_numpy(eq_class)
    eq_class = eq_class.unsqueeze(0)
    
    #create of graph one at the time
    graph = Data(node_features,edge_index,edge_features,y=eq_class)
    
    #print(graph)
    graphs.append(graph)
    
    holder = holder + num_nodes
    edge_holder = edge_holder + edge_amount


    

torch.save(graphs,'data/stdc/d9p1/graphs.pt')#change namnes  


In [4]:
dataset = torch.load('data/stdc/d5p05/graphs_clean.pt')
#print()
#print(f'Dataset: {dataset}:')
#print('====================')
print(f'Number of graphs: {len(dataset)}')
print()
#print(f'Number of features: {dataset.num_features}')
#print(f'Number of classes: {dataset.num_classes}')
print(f'First graph attributes:')
data = dataset[0]  # Get the first graph object.
#print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')


Number of graphs: 4557

First graph attributes:
Data(edge_attr=[12], edge_index=[2, 12], x=[4, 2], y=[1, 4])
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00


In [34]:
torch.manual_seed(1234)
#dataset = dataset.shuffle()
random.shuffle(dataset)

train_dataset = dataset[:8]
test_dataset = dataset[8:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')



Number of training graphs: 8
Number of test graphs: 90
